In [2]:
!pip install transformers[torch] accelerate datasets pandas pypdf2 -U -q
print(" 필수 라이브러리 설치 및 업데이트 완료.")

from google.colab import drive
import torch
import pandas as pd
import PyPDF2
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers.utils import is_torch_available
import os
import re

# Drive 마운트
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/AIHUB_DATA/'
FINAL_MODEL_DIR = BASE_DIR + 'final_nmt_model_1500'
PDF_FILE_PATHS = [
    BASE_DIR + 'lec11_oop.pdf',
    BASE_DIR + 'ml4e-lecture-week13.pdf',
    BASE_DIR + 'NLP_11.pdf'
]

MAX_LENGTH = 50
NUM_BEAMS = 4

USER_START_PAGE = 1
USER_END_PAGE = 5
START_PAGE_INDEX = USER_START_PAGE - 1
PAGE_COUNT = USER_END_PAGE - USER_START_PAGE + 1

model = None
tokenizer = None
if os.path.isdir(FINAL_MODEL_DIR):
    try:
        if is_torch_available() and torch.cuda.is_available():
            device_map = "auto"
            device_info = "GPU"
        else:
            device_map = "cpu"
            device_info = "CPU"

        # 토크나이저 및 모델 로드
        tokenizer = AutoTokenizer.from_pretrained(FINAL_MODEL_DIR)
        model = AutoModelForSeq2SeqLM.from_pretrained(FINAL_MODEL_DIR, device_map=device_map)
        print(f" 모델 로드 완료: {FINAL_MODEL_DIR} (Device: {device_info})")

        # 디코더 시작 토큰 설정
        try:
            KOREAN_BOS_TOKEN_ID = tokenizer.lang_code_to_id["ko_KR"]
            model.config.decoder_start_token_id = KOREAN_BOS_TOKEN_ID
            model.generation_config.forced_bos_token_id = KOREAN_BOS_TOKEN_ID
        except KeyError:
            print(" ko_KR 언어 코드를 찾을 수 없습니다. 모델 설정을 확인하세요.")

    except Exception as e:
        print(f" 모델 로드 오류: {e}. Drive 경로와 저장된 파일({FINAL_MODEL_DIR})을 확인하세요.")
else:
    print(f" 모델 디렉토리를 찾을 수 없습니다: {FINAL_MODEL_DIR}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 151.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
 필수 라이브러리 설치 및 업데이트 완료.
Mounted at /content/drive
 모델 로드 완료: /content/drive/MyDrive/AIHUB_DATA/final_nmt_model_1500 (Device: GPU)


In [3]:
def extract_text_from_pdf_range(pdf_path, start_page_index, num_pages):
    """
    지정된 페이지 범위의 PDF 파일에서 텍스트를 추출하고 문장 단위로 분할합니다.
    """
    raw_text = ""
    if not os.path.exists(pdf_path):
        print(f" 오류: PDF 파일 경로를 찾을 수 없습니다: {pdf_path}")
        return []

    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            end_page_index = start_page_index + num_pages

            for i in range(start_page_index, min(end_page_index, len(reader.pages))):
                raw_text += reader.pages[i].extract_text() or ""

            # 개행 문자를 기준으로 세그먼트 분할 및 빈 줄 제거
            return [raw_text]

    except Exception as e:
        print(f" PDF 처리 중 오류 발생: {e}")
        return []


def generate_translation(texts: list) -> list:
    """훈련된 모델을 사용하여 영어 텍스트를 한국어로 번역하고 결과를 반환합니다."""
    if model is None or tokenizer is None:
        return ["모델 로드 실패"] * len(texts)

    try:
        inputs = tokenizer(texts,
                           return_tensors="pt",
                           padding=True,
                           truncation=True,
                           max_length=MAX_LENGTH)

        # 입력을 모델이 있는 장치(GPU/CPU)로 이동
        input_ids = inputs['input_ids'].to(model.device)
        attention_mask = inputs['attention_mask'].to(model.device)

        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,

            # Generation Parameters
            max_length=MAX_LENGTH,
            num_beams=NUM_BEAMS,
            do_sample=False,
        )

        # 결과 디코딩
        translated_texts = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        return translated_texts
    except Exception as e:
        print(f"추론 중 오류 발생: {e}")
        return ["추론 오류 발생"] * len(texts)

In [9]:
if model is not None and tokenizer is not None:
    all_results = []

    print(f" 다중 PDF 추론 시작: {len(PDF_FILE_PATHS)}개 파일 처리 (Page {USER_START_PAGE} ~ {USER_END_PAGE})")

    for idx, pdf_path in enumerate(PDF_FILE_PATHS):
        file_name = os.path.basename(pdf_path)
        print(f"\n---  {idx+1}/{len(PDF_FILE_PATHS)} 파일 처리 중: {file_name} ---")

        # extract_text_from_pdf_range가 텍스트 전체를 하나의 리스트로 반환하므로 [0] 접근
        raw_texts = extract_text_from_pdf_range(pdf_path, START_PAGE_INDEX, PAGE_COUNT)

        if not raw_texts or not raw_texts[0].strip():
             print(f" {file_name}: 텍스트를 추출하지 못했거나 파일이 비어있습니다.")
             continue

        raw_text = raw_texts[0]

        # 공격적인 클리닝: URL, 이메일, 특수 문자 제거, 개행 문자 공백으로 대체
        cleaned_text = raw_text.replace('\n', ' ').strip() # 모든 개행 문자를 공백으로 대체
        cleaned_text = re.sub(r'http\S+|www\S+|\S+\.pdf|\S+@\S+', '', cleaned_text, flags=re.MULTILINE).strip()
        cleaned_text = re.sub(r'[•*–—]', '', cleaned_text) # Bullet, 대시 등 제거

        # PDF 연결된 텍스트를 분리하기 위한 임시 구분자 삽입:
        temp_separator = ' [SEP] '
        text_with_separators = re.sub(r'([.?!])\s*', r'\1' + temp_separator, cleaned_text)
        # 소문자 뒤에 대문자가 바로 오는 경우 (엉겨 붙은 헤더/목차)를 문장 경계로 간주하여 분리
        text_with_separators = re.sub(r'([a-z])([A-Z])', r'\1' + temp_separator + r'\2', text_with_separators)

        # 2개 이상의 연속된 공백을 1개로 변환
        text_with_separators = re.sub(r'\s{2,}', ' ', text_with_separators)

        #임시 구분자를 포함하여 문장 분할
        segments_split = [s.strip() for s in text_with_separators.split(temp_separator) if s.strip()]

        cleaned_segments = []
        MAX_TOKENS = MAX_LENGTH - 2

        # 분할된 문장들을 필터링 및 길이 제한
        for sentence in segments_split:
            # 1차 필터: 짧은 문장 필터링 (최소 5 단어 미만은 제목/주석일 가능성이 높음)
            if len(sentence.split()) < 5 and not sentence.endswith(('.', '?', '!')):
                continue

            # 2차 필터: 불필요한 기호나 수식이 과도한 문장 필터링 (수식, 코드 제거 목적)
            symbol_count = len(re.findall(r'[()\[\]\{\}<>\/\\=\+\*]', sentence))
            if symbol_count / len(sentence) > 0.15:
                 continue

            # 3차 필터: 토큰 길이 확인 및 자르기
            tokens = tokenizer.tokenize(sentence)
            if len(tokens) > MAX_TOKENS:
                # 50 토큰을 넘는 경우, 자르고 다시 문자열로 변환하여 입력 시퀀스 길이 제한 준수
                tokens = tokens[:MAX_TOKENS]
                sentence = tokenizer.convert_tokens_to_string(tokens).strip()

            if sentence:
                cleaned_segments.append(sentence)

        if cleaned_segments:
            print(f" 총 {len(cleaned_segments)}개의 유효한 문장 세그먼트 추출 완료. 추론 시작.")

            # 추론 실행
            translations = generate_translation(cleaned_segments)

            # 결과 저장
            for src, trg in zip(cleaned_segments, translations):
                all_results.append({
                    "File": file_name,
                    "Source (EN)": src,
                    "Translation (KO)": trg
                })

            print(f" {file_name} 추론 및 결과 저장 완료.")

        else:
            print(f" {file_name}: 정제 후 유효한 문장 세그먼트를 확보하지 못했습니다.")


    if all_results:
        df_report = pd.DataFrame(all_results)

        print(f"최종 추론: 총 {len(df_report)}개 예시 ({len(PDF_FILE_PATHS)}개 파일 사용)")
        print(df_report.to_markdown(index=True))
        print("="*80)
    else:
        print("\n 모든 파일에서 유효한 추론 예시를 확보하지 못했습니다.")
else:
    print("\n 모델 로드가 실패하여 추론을 실행할 수 없습니다. 경로를 확인해주세요.")

 다중 PDF 추론 시작: 3개 파일 처리 (Page 1 ~ 5)

---  1/3 파일 처리 중: lec11_oop.pdf ---
 총 10개의 유효한 문장 세그먼트 추출 완료. 추론 시작.
 lec11_oop.pdf 추론 및 결과 저장 완료.

---  2/3 파일 처리 중: ml4e-lecture-week13.pdf ---
 총 12개의 유효한 문장 세그먼트 추출 완료. 추론 시작.
 ml4e-lecture-week13.pdf 추론 및 결과 저장 완료.

---  3/3 파일 처리 중: NLP_11.pdf ---
 총 20개의 유효한 문장 세그먼트 추출 완료. 추론 시작.
 NLP_11.pdf 추론 및 결과 저장 완료.
최종 추론: 총 42개 예시 (3개 파일 사용)
|    | File                    | Source (EN)                                                                                                                                                                                               | Translation (KO)                                                                                                                                                                                                                      |
|---:|:------------------------|:---------------------------------------------------------------------------------------------------------------------